<a href="https://colab.research.google.com/github/sripriyaagarwal16/Green-Life-Hub/blob/main/newsrecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing the surprise library

In [ ]:
 !pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162734 sha256=9a7852bf8572284edbd2fec2e05acd673d32cfdedbb90a327de8ff7246544f4c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
import pandas as pd


#Dataset Loading

In [ ]:
import pandas as pd

# Attempt to read the file, skipping problematic rows
try:
    behaviors_df = pd.read_csv('behaviors.tsv', delimiter='\t', names=['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions'], error_bad_lines=False)
except pd.errors.ParserError as e:
    print("ParserError:", e)
    print("Skipping problematic rows and attempting to read again...")
    behaviors_df = pd.read_csv('behaviors.tsv', delimiter='\t', names=['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions'], skiprows=[5364, 14443, 19795, 25164, 39546, 46690, 57457, 66485, 70030, 75369, 78878, 86052, 89629, 94931, 101997, 109155, 119856, 123477, 127066, 135959, 139554, 148570, 155714, 161072, 166393])

# Suppress DtypeWarning for mixed data types
pd.options.mode.chained_assignment = None

# Further data preprocessing and modeling...


<ipython-input-3-b686a0583592>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  behaviors_df = pd.read_csv('behaviors.tsv', delimiter='\t', names=['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions'], error_bad_lines=False)


#Cleaning the data

In [ ]:
import pandas as pd

# Define a function to clean the data
def clean_behaviors_data(file_path):
    cleaned_lines = []
    with open(file_path, 'r') as file:
        for line in file:
            fields = line.strip().split('\t')
            if len(fields) >= 5:  # Check if the line has at least 5 fields
                impression_id = fields[0]
                user_id = fields[1]
                time = fields[2]
                history = fields[3]
                impressions = '\t'.join(fields[4:])
                cleaned_line = '\t'.join([impression_id, user_id, time, history, impressions])
                cleaned_lines.append(cleaned_line)
    return cleaned_lines

# Clean the behaviors.tsv file
cleaned_lines = clean_behaviors_data('behaviors.tsv')

# Write the cleaned data to a new file
with open('cleaned_behaviors.tsv', 'w') as file:
    file.write('\n'.join(cleaned_lines))

# Read the cleaned data into a DataFrame, skipping problematic lines
behaviors_df = pd.read_csv('cleaned_behaviors.tsv', delimiter='\t', names=['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions'], error_bad_lines=False)


<ipython-input-4-9a0b65912ad1>:27: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  behaviors_df = pd.read_csv('cleaned_behaviors.tsv', delimiter='\t', names=['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions'], error_bad_lines=False)


In [ ]:

news_df = pd.read_csv('news.tsv', delimiter='\t', names=['News_ID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Title_Entities', 'Abstract_Entities'])


In [ ]:
behaviors_df['Time'] = pd.to_datetime(behaviors_df['Time'])

# Example: Extract day of the week from 'Time' column
behaviors_df['Day_of_Week'] = behaviors_df['Time'].dt.day_name()

# Example: Calculate the total number of impressions for each user
impressions_count = behaviors_df.groupby('User_ID')['Impression_ID'].count().reset_index()
impressions_count.columns = ['User_ID', 'Total_Impressions']


In [ ]:
# Convert the 'History' column to strings (if not already)
behaviors_df['History'] = behaviors_df['History'].astype(str)

# Split the 'History' column by spaces to get a list of news article IDs
behaviors_df['History'] = behaviors_df['History'].str.split()

# Count the number of interactions for each news article ID
behaviors_df['Rating'] = behaviors_df['History'].apply(len)

# Convert the 'Rating' column to float
behaviors_df['Rating'] = behaviors_df['Rating'].astype(float)


#Training the data

In [ ]:
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(behaviors_df[['User_ID', 'Impression_ID','Rating']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Train the SVD model (or any other collaborative filtering model)
model = SVD()
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

#Accuracy of Model

In [ ]:
from surprise.accuracy import rmse
from surprise import accuracy

In [ ]:
rmse_score = accuracy.rmse(predictions)


RMSE: 50.3255


In [ ]:
user_id = 'U13740'
items_already_rated_by_user = behaviors_df[behaviors_df['User_ID'] == user_id]['Impression_ID'].tolist()

#Testing

Example1

In [ ]:
all_impression_ids = behaviors_df['Impression_ID'].unique()
impressions_to_predict = [impression_id for impression_id in all_impression_ids if impression_id not in items_already_rated_by_user]
user_predictions = [model.predict(user_id, impression_id) for impression_id in impressions_to_predict]
user_predictions.sort(key=lambda x: x.est, reverse=True)
top_n = 10
recommended_impression_ids = [prediction.iid for prediction in user_predictions[:top_n]]
print("Top", top_n, "recommended impressions for user", user_id, ":", recommended_impression_ids)

Top 10 recommended impressions for user U13740 : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the News_ID column
news_df['Encoded_News_ID'] = label_encoder.fit_transform(news_df['News_ID'])

# Print the encoded news IDs
print(news_df[['News_ID', 'Encoded_News_ID']])


      News_ID  Encoded_News_ID
0      N55528            10749
1      N19639             2300
2      N61837            12256
3      N53526            10271
4      N38324             6770
...       ...              ...
13869  N58119            11350
13870  N26928             4068
13871  N51841             9881
13872  N37632             6593
13873  N43654             8004

[13874 rows x 2 columns]


In [ ]:
def get_news_details(impression_ids):
    news_details = []
    for impression_id in impression_ids:
        # Find the corresponding news article based on impression ID
        news_article = news_df.loc[news_df['Encoded_News_ID'] == impression_id]
        if not news_article.empty:
            # Extract relevant information
            news_title = news_article['Title'].values[0]
            news_category = news_article['Category'].values[0]
            news_abstract = news_article['Abstract'].values[0]
            news_url = news_article['URL'].values[0]
            # Append news details to the list
            news_details.append({'Impression_ID': impression_id,
                                 'Title': news_title,
                                 'Category': news_category,
                                 'Abstract': news_abstract,
                                 'URL': news_url})
            # Print the statement for the recommended news
            print("The recommended news for the impression " + str(impression_id) + " is:")
            print("Title:", news_title)
            print("Category:", news_category)
            print("Abstract:", news_abstract)
            print("URL:", news_url)
            print()  # Print an empty line for readability
        else:
            print(f"No news article found for impression ID: {impression_id}")
    return news_details


In [ ]:
get_news_details( [2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

The recommended news for the impression 2 is:
Title: The one and only Chevy eCOPO Camaro electric drag-racer is for sale
Category: autos
Abstract: After serving its life as a development vehicle, the eCOPO is off to find a new home.
URL: https://assets.msn.com/labs/mind/AAFI2kD.html

The recommended news for the impression 3 is:
Title: Eagles vs. Cowboys: Second quarter open thread
Category: sports
Abstract: After the first quarter, the Eagles trail the Cowboys by a score of 14 to 7.
URL: https://assets.msn.com/labs/mind/AAJ5bMr.html

The recommended news for the impression 4 is:
Title: Bicyclist dies after being hit by vehicle in Brookline
Category: news
Abstract: A bicyclist who was hit by a vehicle Sunday in Brookline has died as a result of his wounds, police said Monday. The bicyclist was struck by a vehicle after 9:15 p.m. at Brookline Boulevard and Pioneer Avenue. The bicyclist, a male, was unconscious when first responders arrived and was taken to UPMC Mercy hospital in critica

[{'Impression_ID': 2,
  'Title': 'The one and only Chevy eCOPO Camaro electric drag-racer is for sale',
  'Category': 'autos',
  'Abstract': 'After serving its life as a development vehicle, the eCOPO is off to find a new home.',
  'URL': 'https://assets.msn.com/labs/mind/AAFI2kD.html'},
 {'Impression_ID': 3,
  'Title': 'Eagles vs. Cowboys: Second quarter open thread',
  'Category': 'sports',
  'Abstract': 'After the first quarter, the Eagles trail the Cowboys by a score of 14 to 7.',
  'URL': 'https://assets.msn.com/labs/mind/AAJ5bMr.html'},
 {'Impression_ID': 4,
  'Title': 'Bicyclist dies after being hit by vehicle in Brookline',
  'Category': 'news',
  'Abstract': 'A bicyclist who was hit by a vehicle Sunday in Brookline has died as a result of his wounds, police said Monday. The bicyclist was struck by a vehicle after 9:15 p.m. at Brookline Boulevard and Pioneer Avenue. The bicyclist, a male, was unconscious when first responders arrived and was taken to UPMC Mercy hospital in crit

Example2

In [ ]:
user_id_1 = 'U12957'
items_already_rated_by_user = behaviors_df[behaviors_df['User_ID'] == user_id_1]['Impression_ID'].tolist()

In [ ]:
all_impression_ids = behaviors_df['Impression_ID'].unique()
impressions_to_predict = [impression_id for impression_id in all_impression_ids if impression_id not in items_already_rated_by_user]
user_predictions = [model.predict(user_id_1, impression_id) for impression_id in impressions_to_predict]
user_predictions.sort(key=lambda x: x.est, reverse=True)
top_n = 10
recommended_impression_ids = [prediction.iid for prediction in user_predictions[:top_n]]
print("Top", top_n, "recommended impressions for user", user_id_1, ":", recommended_impression_ids)

Top 10 recommended impressions for user U12957 : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
get_news_details( [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

The recommended news for the impression 1 is:
Title: Parents capture boy's stunned reaction to becoming a big brother
Category: video
Abstract: Sometimes the best gifts are those you're least expecting, especially for this young boy whose parents surprised him with a book titled "Being a Big Brother." It took him a few seconds to let it all sink in, but the reaction was priceless.
URL: https://assets.msn.com/labs/mind/AAIoP4y.html

The recommended news for the impression 2 is:
Title: The one and only Chevy eCOPO Camaro electric drag-racer is for sale
Category: autos
Abstract: After serving its life as a development vehicle, the eCOPO is off to find a new home.
URL: https://assets.msn.com/labs/mind/AAFI2kD.html

The recommended news for the impression 3 is:
Title: Eagles vs. Cowboys: Second quarter open thread
Category: sports
Abstract: After the first quarter, the Eagles trail the Cowboys by a score of 14 to 7.
URL: https://assets.msn.com/labs/mind/AAJ5bMr.html

The recommended news fo

[{'Impression_ID': 1,
  'Title': "Parents capture boy's stunned reaction to becoming a big brother",
  'Category': 'video',
  'Abstract': 'Sometimes the best gifts are those you\'re least expecting, especially for this young boy whose parents surprised him with a book titled "Being a Big Brother." It took him a few seconds to let it all sink in, but the reaction was priceless.',
  'URL': 'https://assets.msn.com/labs/mind/AAIoP4y.html'},
 {'Impression_ID': 2,
  'Title': 'The one and only Chevy eCOPO Camaro electric drag-racer is for sale',
  'Category': 'autos',
  'Abstract': 'After serving its life as a development vehicle, the eCOPO is off to find a new home.',
  'URL': 'https://assets.msn.com/labs/mind/AAFI2kD.html'},
 {'Impression_ID': 3,
  'Title': 'Eagles vs. Cowboys: Second quarter open thread',
  'Category': 'sports',
  'Abstract': 'After the first quarter, the Eagles trail the Cowboys by a score of 14 to 7.',
  'URL': 'https://assets.msn.com/labs/mind/AAJ5bMr.html'},
 {'Impress